In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import plotly.express as px
!pip install scikit-optimize
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import math
!pip install catboost --upgrade
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import os
import warnings
import numpy as np  
import seaborn as sns
import pandas as pd, os, gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
warnings.filterwarnings('ignore')
from typing import List

In [ ]:
#Importing data
train=pd.read_csv('/content/drive/My Drive/Train (1).csv')
test=pd.read_csv('/content/drive/My Drive/Test (1).csv')

In [ ]:
true_values = []
for v in test.values:
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in test.columns[8:][index]:
    true_values.append(v[0] + ' X ' + k)

In [ ]:
#Store Products ID in Multidimensional array
products =train[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
products_t =test[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]

#Axis Transformation
train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )

#A merger
train['combiner']='x'

#Set
data=pd.concat([train,test],sort=False).reset_index(drop=True)

train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
train.drop('combiner', inplace=True, axis=1);test.drop(['Label','combiner'], inplace=True, axis=1)

#Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE'];test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

#Re-arrange column (not necessary)
train=train[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE',  'Label']]
test=test[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE']]

In [ ]:
train['date1'] = train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
train['date2'] = train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
train['date3'] = train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
train.drop('join_date', axis=1, inplace=True)

test['date1'] = test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
test['date2'] = test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
test['date3'] = test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
test.drop('join_date', axis=1, inplace=True)

train['date_diff'] = train['date3'] - train['birth_year']
test['date_diff'] = test['date3'] - test['birth_year']

In [ ]:
train

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label,date1,date2,date3,date_diff
0,4WKQSBB,F,M,1987,1X1H,2A7I,T4MS,P5DA,4WKQSBB X P5DA,0,1.0,2.0,2019.0,32.0
1,CP5S02H,F,M,1981,UAOD,2A7I,T4MS,P5DA,CP5S02H X P5DA,0,1.0,6.0,2019.0,38.0
2,2YKDILJ,M,U,1991,748L,QZYX,90QI,P5DA,2YKDILJ X P5DA,0,1.0,6.0,2013.0,22.0
3,2S9E81J,M,M,1990,1X1H,BP09,56SI,P5DA,2S9E81J X P5DA,0,1.0,8.0,2019.0,29.0
4,BHDYVFT,M,M,1990,748L,NO3L,T4MS,P5DA,BHDYVFT X P5DA,0,1.0,8.0,2019.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611767,8SBA93U,F,M,1974,748L,2A7I,T4MS,ECY3,8SBA93U X ECY3,0,1.0,9.0,2019.0,45.0
611768,Y73ETTO,M,M,1987,XX25,Q6J6,90QI,ECY3,Y73ETTO X ECY3,0,1.0,11.0,2018.0,31.0
611769,MVXG2LC,M,M,1985,BOAS,6PE7,T4MS,ECY3,MVXG2LC X ECY3,0,1.0,2.0,2019.0,34.0
611770,BUQ8DQN,M,M,1989,UAOD,2A7I,T4MS,ECY3,BUQ8DQN X ECY3,0,1.0,7.0,2019.0,30.0


In [ ]:
test

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,date1,date2,date3,date_diff
0,F86J5PC,M,M,1984,94KC,DZRV,90QI,P5DA,F86J5PC X P5DA,1.0,12.0,2018.0,34.0
1,H6141K3,M,M,1996,1X1H,J9SY,90QI,P5DA,H6141K3 X P5DA,1.0,10.0,2019.0,23.0
2,RBAYUXZ,F,W,1968,UAOD,2A7I,T4MS,P5DA,RBAYUXZ X P5DA,1.0,1.0,2020.0,52.0
3,KCBILBQ,M,M,1989,94KC,2A7I,T4MS,P5DA,KCBILBQ X P5DA,1.0,2.0,2019.0,30.0
4,LSEC1ZJ,F,M,1982,UAOD,0KID,T4MS,P5DA,LSEC1ZJ X P5DA,1.0,2.0,2020.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,0GMU5UH,F,M,1986,748L,2A7I,T4MS,ECY3,0GMU5UH X ECY3,1.0,4.0,2019.0,33.0
209996,I9W11CD,M,M,1980,UAOD,0KID,T4MS,ECY3,I9W11CD X ECY3,1.0,8.0,2019.0,39.0
209997,42WTEGT,M,M,1993,748L,SST3,56SI,ECY3,42WTEGT X ECY3,1.0,4.0,2018.0,25.0
209998,8EKC4O9,F,U,1981,49BM,2A7I,T4MS,ECY3,8EKC4O9 X ECY3,1.0,5.0,2015.0,34.0


In [ ]:
catFeatureslist=train[['occupation_category_code','occupation_code','branch_code','marital_status','sex','PCODE']]
catFeatureslist1=test[['occupation_category_code','occupation_code','branch_code','marital_status','sex','PCODE']]

In [ ]:
for cf1 in catFeatureslist:
    le = LabelEncoder()
    le.fit(train[cf1].unique())
    train[cf1] = le.transform(train[cf1])

In [ ]:
train

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label,date1,date2,date3,date_diff
0,4WKQSBB,0,1,0.500000,0,17,5,15,4WKQSBB X P5DA,0,0.0,-0.8,0.5,-0.428571
1,CP5S02H,0,1,0.071429,11,17,5,15,CP5S02H X P5DA,0,0.0,0.0,0.5,0.000000
2,2YKDILJ,1,5,0.785714,3,176,1,15,2YKDILJ X P5DA,0,0.0,0.0,-2.5,-1.142857
3,2S9E81J,1,1,0.714286,0,76,0,15,2S9E81J X P5DA,0,0.0,0.4,0.5,-0.642857
4,BHDYVFT,1,1,0.714286,3,148,5,15,BHDYVFT X P5DA,0,0.0,0.4,0.5,-0.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611767,8SBA93U,0,1,-0.428571,3,17,5,5,8SBA93U X ECY3,0,0.0,0.6,0.5,0.500000
611768,Y73ETTO,1,1,0.500000,13,170,1,5,Y73ETTO X ECY3,0,0.0,1.0,0.0,-0.500000
611769,MVXG2LC,1,1,0.357143,6,44,5,5,MVXG2LC X ECY3,0,0.0,-0.8,0.5,-0.285714
611770,BUQ8DQN,1,1,0.642857,11,17,5,5,BUQ8DQN X ECY3,0,0.0,0.2,0.5,-0.571429


In [ ]:
for cf1 in catFeatureslist1:
    le = LabelEncoder()
    le.fit(test[cf1].unique())
    test[cf1] = le.transform(test[cf1])

In [ ]:
test

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,date1,date2,date3,date_diff
0,F86J5PC,1,2,0.285714,4,79,1,15,F86J5PC X P5DA,0.0,1.2,0.0,-0.214286
1,H6141K3,1,2,1.142857,0,103,1,15,H6141K3 X P5DA,0.0,0.8,0.5,-1.000000
2,RBAYUXZ,0,7,-0.857143,11,18,5,15,RBAYUXZ X P5DA,0.0,-1.0,1.0,1.071429
3,KCBILBQ,1,2,0.642857,4,18,5,15,KCBILBQ X P5DA,0.0,-0.8,0.5,-0.500000
4,LSEC1ZJ,0,2,0.142857,11,3,5,15,LSEC1ZJ X P5DA,0.0,-0.8,1.0,0.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,0GMU5UH,0,2,0.428571,3,18,5,5,0GMU5UH X ECY3,0.0,-0.4,0.5,-0.285714
209996,I9W11CD,1,2,0.000000,11,3,5,5,I9W11CD X ECY3,0.0,0.4,0.5,0.142857
209997,42WTEGT,1,2,0.928571,3,156,0,5,42WTEGT X ECY3,0.0,-0.4,0.0,-0.857143
209998,8EKC4O9,0,6,0.071429,2,18,5,5,8EKC4O9 X ECY3,0.0,-0.2,-1.5,-0.214286


In [ ]:
'''# retrieve just the numeric input values
num_cols = ['birth_year','date1','date2','date3','date_diff']'''

In [ ]:
'''# perform a robust scaler transform of the dataset
trans = RobustScaler()
train[num_cols] = trans.fit_transform(train[num_cols])'''

In [ ]:
'''# perform a robust scaler transform of the dataset
trans = RobustScaler()
test[num_cols] = trans.fit_transform(test[num_cols])'''

In [ ]:
train

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label,date1,date2,date3,date_diff
0,4WKQSBB,0,1,0.500000,0,17,5,15,4WKQSBB X P5DA,0,0.0,-0.8,0.5,-0.428571
1,CP5S02H,0,1,0.071429,11,17,5,15,CP5S02H X P5DA,0,0.0,0.0,0.5,0.000000
2,2YKDILJ,1,5,0.785714,3,176,1,15,2YKDILJ X P5DA,0,0.0,0.0,-2.5,-1.142857
3,2S9E81J,1,1,0.714286,0,76,0,15,2S9E81J X P5DA,0,0.0,0.4,0.5,-0.642857
4,BHDYVFT,1,1,0.714286,3,148,5,15,BHDYVFT X P5DA,0,0.0,0.4,0.5,-0.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611767,8SBA93U,0,1,-0.428571,3,17,5,5,8SBA93U X ECY3,0,0.0,0.6,0.5,0.500000
611768,Y73ETTO,1,1,0.500000,13,170,1,5,Y73ETTO X ECY3,0,0.0,1.0,0.0,-0.500000
611769,MVXG2LC,1,1,0.357143,6,44,5,5,MVXG2LC X ECY3,0,0.0,-0.8,0.5,-0.285714
611770,BUQ8DQN,1,1,0.642857,11,17,5,5,BUQ8DQN X ECY3,0,0.0,0.2,0.5,-0.571429


In [ ]:
test

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,date1,date2,date3,date_diff
0,F86J5PC,1,2,0.285714,4,79,1,15,F86J5PC X P5DA,0.0,1.2,0.0,-0.214286
1,H6141K3,1,2,1.142857,0,103,1,15,H6141K3 X P5DA,0.0,0.8,0.5,-1.000000
2,RBAYUXZ,0,7,-0.857143,11,18,5,15,RBAYUXZ X P5DA,0.0,-1.0,1.0,1.071429
3,KCBILBQ,1,2,0.642857,4,18,5,15,KCBILBQ X P5DA,0.0,-0.8,0.5,-0.500000
4,LSEC1ZJ,0,2,0.142857,11,3,5,15,LSEC1ZJ X P5DA,0.0,-0.8,1.0,0.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,0GMU5UH,0,2,0.428571,3,18,5,5,0GMU5UH X ECY3,0.0,-0.4,0.5,-0.285714
209996,I9W11CD,1,2,0.000000,11,3,5,5,I9W11CD X ECY3,0.0,0.4,0.5,0.142857
209997,42WTEGT,1,2,0.928571,3,156,0,5,42WTEGT X ECY3,0.0,-0.4,0.0,-0.857143
209998,8EKC4O9,0,6,0.071429,2,18,5,5,8EKC4O9 X ECY3,0.0,-0.2,-1.5,-0.214286


In [ ]:
X=train.drop(columns=['ID', 'Label', 'ID X PCODE'],axis=1)

In [ ]:
y_target=train['Label'].apply(lambda x:int(x)).values

In [ ]:
y=train['Label']

In [ ]:
X_test=test.drop(columns=['ID', 'ID X PCODE'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Model selection
from sklearn.model_selection import StratifiedKFold

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from time import time

In [ ]:
def report_perf(optimizer, X, y, title, callbacks=None):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time()
    if callbacks:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start, 
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

In [ ]:
import pprint
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib


In [ ]:
roc_auc = make_scorer(roc_auc_score, greater_is_better=True, needs_threshold=True)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
clf = CatBoostClassifier(thread_count=2,
                         loss_function='Logloss',
                        
                         od_type = 'Iter',
                         verbose= False
                        )

In [ ]:
# Defining your search space
search_spaces = {'iterations': Integer(10, 1000),
                 'depth': Integer(1, 8),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 'random_strength': Real(1e-9, 10, 'log-uniform'),
                 'bagging_temperature': Real(0.0, 1.0),
                 'border_count': Integer(1, 255),
                 'l2_leaf_reg': Integer(2, 30),
                 'scale_pos_weight':Real(0.01, 1.0, 'uniform')}

In [ ]:
# Setting up BayesSearchCV
opt = BayesSearchCV(clf,
                    search_spaces,
                    scoring=roc_auc,
                    cv=skf,
                    n_iter=100,
                    n_jobs=1,  # use just 1 job with CatBoost in order to avoid segmentation fault
                    return_train_score=False,
                    refit=True,
                    optimizer_kwargs={'base_estimator': 'GP'},
                    random_state=42)

In [ ]:
best_params = report_perf(opt, X, y_target,'CatBoost',
                          callbacks=[VerboseCallback(100), 
                                  DeadlineStopper(60*10)])

Iteration No: 1 started. Searching for the next optimal point.
Iteration No: 1 ended. Search finished for the next optimal point.
Time taken: 312.3362
Function value obtained: -0.9737
Current minimum: -0.9737
Iteration No: 2 started. Searching for the next optimal point.
CatBoost took 389.35 seconds,  candidates checked: 1, best CV score: 0.974 ± 0.001
Best parameters:
OrderedDict([('bagging_temperature', 0.41010395885331385),
             ('border_count', 186),
             ('depth', 8),
             ('iterations', 323),
             ('l2_leaf_reg', 21),
             ('learning_rate', 0.0673344419215237),
             ('random_strength', 3.230824361824754e-06),
             ('scale_pos_weight', 0.7421091918485163)])



In [ ]:
best_params={'bagging_temperature': 0.41010395885331385,
 'border_count': 186,
 'depth': 8,
 'iterations': 323,
 'l2_leaf_reg': 21,
 'learning_rate': 0.0673344419215237,
 'random_strength': 3.230824361824754e-06,
 'scale_pos_weight': 0.7421091918485163}

In [ ]:
best_params['iterations']=1000

In [ ]:
from time import time

In [ ]:
%%time 
 tuned_model = CatBoostClassifier(**best_params,task_type = "CPU",od_type='Iter',one_hot_max_size=10)
 tuned_model.fit(X,y)

0:	learn: 0.5757061	total: 245ms	remaining: 4m 5s
1:	learn: 0.4827658	total: 484ms	remaining: 4m 1s
2:	learn: 0.4124514	total: 722ms	remaining: 3m 59s
3:	learn: 0.3547676	total: 959ms	remaining: 3m 58s
4:	learn: 0.3112140	total: 1.22s	remaining: 4m 1s
5:	learn: 0.2757251	total: 1.46s	remaining: 4m 1s
6:	learn: 0.2475061	total: 1.7s	remaining: 4m
7:	learn: 0.2261627	total: 1.93s	remaining: 3m 59s
8:	learn: 0.2073664	total: 2.18s	remaining: 3m 59s
9:	learn: 0.1963152	total: 2.42s	remaining: 3m 59s
10:	learn: 0.1845430	total: 2.66s	remaining: 3m 59s
11:	learn: 0.1749390	total: 2.9s	remaining: 3m 58s
12:	learn: 0.1661078	total: 3.14s	remaining: 3m 58s
13:	learn: 0.1597085	total: 3.38s	remaining: 3m 57s
14:	learn: 0.1535530	total: 3.62s	remaining: 3m 57s
15:	learn: 0.1497968	total: 3.85s	remaining: 3m 56s
16:	learn: 0.1453389	total: 4.1s	remaining: 3m 57s
17:	learn: 0.1425295	total: 4.34s	remaining: 3m 56s
18:	learn: 0.1397224	total: 4.57s	remaining: 3m 55s
19:	learn: 0.1378487	total: 4.81s

In [ ]:
probaaa = tuned_model.predict_proba(X_test)[:,1]

In [ ]:
probaaa

array([0.00041307, 0.0001802 , 0.00299684, ..., 0.00510242, 0.01375556,
       0.03310414])

In [ ]:
log_loss(y_test,probaaa)

In [ ]:
df_answer = test[['ID X PCODE']]
df_answer['Label'] = probaaa

In [ ]:
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
  if df_answer['ID X PCODE'].iloc[i] in true_values:
    df_answer['Label'].iloc[i] = 1.0

df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission7.csv', index=False)